In [2]:
import numpy as np
import random
from sklearn import svm
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the Diabetes dataset
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the ICA function for feature selection
def initialize_countries(num_countries, num_features):
    countries = []
    for _ in range(num_countries):
        country = [random.randint(0, 1) for _ in range(num_features)]
        countries.append(country)
    return countries

def calculate_cost(country, X_train, y_train, X_test, y_test):
    selected_features = [i for i, x in enumerate(country) if x == 1]
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]
    model = svm.SVR()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    cost = np.mean((y_pred - y_test) ** 2)
    return cost

def imperialist_competitive_algorithm(num_countries, num_features, X_train, y_train, X_test, y_test, max_iterations):
    countries = initialize_countries(num_countries, num_features)
    costs = [calculate_cost(country, X_train, y_train, X_test, y_test) for country in countries]
    empires = []
    colonies = []

    for _ in range(max_iterations):
        # Sort countries based on their costs
        sorted_indices = np.argsort(costs)
        countries = [countries[i] for i in sorted_indices]
        costs = [costs[i] for i in sorted_indices]

        # Create empires
        empires = []
        for i in range(num_countries // 2):
            empire = {'imperialist': countries[i], 'colonies': []}
            empires.append(empire)

        # Assign colonies to empires
        colonies = countries[num_countries // 2:]
        for colony in colonies:
            # Randomly select an empire and add the colony to it
            empire_index = random.randint(0, len(empires) - 1)
            empires[empire_index]['colonies'].append(colony)

        # Move colonies towards their imperialist
        for empire in empires:
            for colony in empire['colonies']:
                # Randomly select a feature and change it to match the imperialist
                feature_index = random.randint(0, num_features - 1)
                colony[feature_index] = empire['imperialist'][feature_index]

        # Calculate new costs for colonies
        new_costs = []
        for empire in empires:
            for colony in empire['colonies']:
                new_cost = calculate_cost(colony, X_train, y_train, X_test, y_test)
                new_costs.append(new_cost)

        # Update colonies with new costs
        colonies = [country for country, cost in zip(colonies, new_costs) if cost < calculate_cost(country, X_train, y_train, X_test, y_test)]

        # Update empires with new colonies
        for empire in empires:
            empire['colonies'] = colonies

        # Update costs
        costs = [calculate_cost(country, X_train, y_train, X_test, y_test) for country in countries]

    # Return the best country
    best_country_index = np.argmin(costs)
    return countries[best_country_index]

# Define the parameters for the ICA
num_countries = 10
num_features = X_train.shape[1]
max_iterations = 100

# Run the ICA for feature selection
best_country = imperialist_competitive_algorithm(num_countries, num_features, X_train, y_train, X_test, y_test, max_iterations)

# Print the selected features
selected_features = [i for i, x in enumerate(best_country) if x == 1]
print("Selected features:", selected_features)

# Evaluate the model with the selected features
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]
model = svm.SVR()
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)
print("MSE:", np.mean((y_pred - y_test) ** 2))


Selected features: [2, 8]
MSE: 3620.950819473882
